# Create the Fetures Group from PCAPs

In [1]:

# scapy verification
try:
    import scapy
except ImportError:
    print("Installation of scapy")
    !pip install scapy

import numpy as np
import pandas as pd
from scapy.all import *
from datetime import datetime
import string
import re

# wget package
try:
    import wget
except ImportError:
    print("Installation of wget")
    !pip install wget


**Load PCAP Data:**

In [2]:
def load_pcap_data(filepath):

    packets = rdpcap(filepath)
    print(f"Number of packets in test load: {len(packets)}")
    if packets:
        print(packets[0].summary())
    return packets


**Extract Network Traffic Features:**

In [3]:
def extract_network_traffic_features(packets):
    network_traffic_features = []

    import numpy as np
    import pandas as pd

    for packet in packets:
        # Initialize variables for each feature
        eth_src = None
        eth_dst = None
        src_ip = None
        dst_ip = None
        src_port = None
        dst_port = None
        protocol = None
        timestamp = packet.time

        # Extract Ethernet frame information
        if Ether in packet:
            eth_src = packet[Ether].src
            eth_dst = packet[Ether].dst

        # Extract IP packet information
        if IP in packet:
            src_ip = packet[IP].src
            dst_ip = packet[IP].dst

            if TCP in packet:
                src_port = packet[TCP].sport
                dst_port = packet[TCP].dport
                protocol = 'TCP'
            elif UDP in packet:
                src_port = packet[UDP].sport
                dst_port = packet[UDP].dport
                protocol = 'UDP'
            else:
                # Handle other IP protocols as needed
                protocol = packet[IP].proto

        # Append extracted features as a dictionary to the network_traffic_features list
        network_traffic_features.append({
            'eth_src': eth_src,
            'eth_dst': eth_dst,
            'src_ip': src_ip,
            'dst_ip': dst_ip,
            'src_port': src_port,
            'dst_port': dst_port,
            'protocol': protocol,
            'timestamp': timestamp
        })

    return pd.DataFrame(network_traffic_features)


 **Extract Session-related Information Features:**

In [4]:
def extract_session_behavior_features(packets):
    session_behaviour_features = []
    import numpy as np
    import pandas as pd
    from datetime import datetime
    from scapy.all import Ether, IP

    current_session_start_time = None
    current_session_end_time = None
    session_start_time = None
    failed_login_count = 0
    session_behaviour_features = []
    session_timestamps = []

    # Define a pattern to identify authentication-related events
    auth_event_pattern = "authentication_failed"

    # Iterate through the packets and extract the features
    for packet in packets:
        # Extract Ethernet frame information
        if Ether in packet:
            eth_src = packet[Ether].src
            eth_dst = packet[Ether].dst

        # Extract IP packet information
        if IP in packet:
            # Update session start and end times
            if current_session_start_time is None:
                current_session_start_time = packet.time
                # Convert packet.time to float
                timestamp = datetime.utcfromtimestamp(float(packet.time)).strftime('%Y-%m-%d %H:%M:%S')
                session_timestamps.append(timestamp)
            current_session_end_time = packet.time

            # Check if the packet contains the authentication event pattern
            if auth_event_pattern in str(packet.payload):
                failed_login_count += 1
        else:
            # End of session or non-IP packet
            if current_session_start_time is not None:
                # Calculate session duration
                session_duration = current_session_end_time - current_session_start_time

                # Append session behavior features to the list
                session_behaviour_features.append([session_duration, 1, failed_login_count])

                # Calculate interval between authentication attempts
                if session_start_time is not None:
                    auth_attempt_interval = current_session_start_time - session_start_time
                    session_behaviour_features[-1].append(auth_attempt_interval)
                else:
                    session_behaviour_features[-1].append(None)

                # Reset session-related variables for the next session
                current_session_start_time = None
                current_session_end_time = None
                session_start_time = packet.time
                failed_login_count = 0

    return pd.DataFrame(session_behaviour_features, columns=['session_duration', 'auth_event_count', 'failed_login_count', 'auth_attempt_interval'])

**Extract Network Flow Features:**

In [5]:
def extract_network_flow_features(packets):
    network_flow_features = []
    import numpy as np
    import pandas as pd
    from scapy.all import Ether, IP, TCP, UDP
    from datetime import datetime

    # Define dictionaries to keep track of flow statistics
    flow_packet_counts = {}
    flow_total_bytes = {}
    flow_first_timestamp = {}
    flow_last_timestamp = {}

    for packet in packets:
        timestamp = datetime.utcfromtimestamp(float(packet.time)).strftime('%Y-%m-%d %H:%M:%S')

        # Extract Ethernet frame information
        if Ether in packet:
            eth_src = packet[Ether].src
            eth_dst = packet[Ether].dst
        else:
            eth_src = None
            eth_dst = None

        # Extract IP packet information
        if IP in packet:
            src_ip = packet[IP].src
            dst_ip = packet[IP].dst

            src_port, dst_port = None, None  # Initialize as None by default

            if TCP in packet:
                src_port = packet[TCP].sport
                dst_port = packet[TCP].dport
            elif UDP in packet:
                src_port = packet[UDP].sport
                dst_port = packet[UDP].dport

            # Define a flow identifier based on source and destination IP addresses and ports
            flow_identifier = (src_ip, dst_ip, src_port, dst_port)

            # Update flow statistics
            if flow_identifier not in flow_packet_counts:
                flow_packet_counts[flow_identifier] = 0
                flow_total_bytes[flow_identifier] = 0
                flow_first_timestamp[flow_identifier] = timestamp  # Initialize first timestamp

            flow_packet_counts[flow_identifier] += 1
            flow_total_bytes[flow_identifier] += len(packet)
            flow_last_timestamp[flow_identifier] = timestamp  # Update last timestamp

    # Extracted network flow features
    network_flow_features = []

    for flow_identifier, packet_count in flow_packet_counts.items():
        src_ip, dst_ip, src_port, dst_port = flow_identifier
        total_bytes = flow_total_bytes[flow_identifier]
        first_timestamp = flow_first_timestamp[flow_identifier]
        last_timestamp = flow_last_timestamp[flow_identifier]

        # Determine traffic direction based on source and destination IP addresses
        if src_ip.startswith('192.168.1'):  # Example: Check if source IP belongs to your network
            traffic_direction = 'Outbound'
        elif dst_ip.startswith('192.168.1'):  # Example: Check if destination IP belongs to your network
            traffic_direction = 'Inbound'
        else:
            traffic_direction = 'External'

        # Append flow features to the network_flow_features list
        network_flow_features.append([src_ip, dst_ip, src_port, dst_port, packet_count, total_bytes, traffic_direction, first_timestamp, last_timestamp])

    # Define the column headers
    headers = ['src_ip', 'dst_ip', 'src_port', 'dst_port', 'packet_count', 'total_bytes', 'traffic_direction', 'first_timestamp', 'last_timestamp']

    return pd.DataFrame(network_flow_features, columns=['src_ip', 'dst_ip', 'src_port', 'dst_port', 'packet_count', 'total_bytes', 'traffic_direction', 'first_timestamp', 'last_timestamp'])


**Extract Protocol-specific Features:**

In [6]:
def extract_protocol_specific_features(packets):
    protocol_specific_features = []
    import numpy as np
    import pandas as pd
    from scapy.all import Ether, IP, TCP, Raw
    from datetime import datetime
    import re

    protocol_specific_features = []

    for packet in packets:
        # Define a dictionary to store protocol-specific features for this packet
        packet_features = {}

        # Extract the timestamp of the packet
        packet_features['timestamp'] = datetime.utcfromtimestamp(float(packet.time)).strftime('%Y-%m-%d %H:%M:%S')

        # Extract Ethernet frame information
        if Ether in packet:
            packet_features['eth_src'] = packet[Ether].src
            packet_features['eth_dst'] = packet[Ether].dst

        # Extract IP packet information
        if IP in packet:
            packet_features['src_ip'] = packet[IP].src
            packet_features['dst_ip'] = packet[IP].dst
            if TCP in packet:
                packet_features['src_port'] = packet[TCP].sport
                packet_features['dst_port'] = packet[TCP].dport

                # Check if the packet is HTTP (port 80)
                if packet_features['src_port'] == 80 or packet_features['dst_port'] == 80:
                    if Raw in packet:
                        http_data = str(packet[Raw].load)

                        # Extract HTTP headers (assuming headers are in the first part of the payload)
                        http_headers = http_data.split('\r\n\r\n')[0]
                        packet_features['http_headers'] = http_headers

                        # Extract HTTP status code (if available)
                        status_code_match = re.search(r'HTTP/1.[01] (\d{3})', http_headers)
                        if status_code_match:
                            packet_features['http_status_code'] = int(status_code_match.group(1))

                # Check if the packet is SSH (port 22)
                if packet_features['src_port'] == 22 or packet_features['dst_port'] == 22:
                    if Raw in packet:
                        ssh_payload = str(packet[Raw].load)
                        # Extract SSH protocol details as needed
                        packet_features['ssh_protocol_details'] = ssh_payload

        # Append the dictionary of features to the list
        protocol_specific_features.append(packet_features)

    return pd.DataFrame(protocol_specific_features)


**Extract Payload Characteristics Features:**

In [7]:
def extract_payload_characteristics_features(packets):
    payload_characteristics_features = []
    import numpy as np
    import pandas as pd
    from scapy.all import Ether, IP, TCP, Raw
    from datetime import datetime
    import string

    payload_characteristics_features = []

    for packet in packets:
        # Define a dictionary to store payload characteristics features for this packet
        packet_features = {}

        # Extract the timestamp of the packet
        packet_features['timestamp'] = datetime.utcfromtimestamp(float(packet.time)).strftime('%Y-%m-%d %H:%M:%S')

        # Extract Ethernet frame information
        if Ether in packet:
            packet_features['eth_src'] = packet[Ether].src
            packet_features['eth_dst'] = packet[Ether].dst

        # Extract IP packet information
        if IP in packet:
            packet_features['src_ip'] = packet[IP].src
            packet_features['dst_ip'] = packet[IP].dst
            if TCP in packet:
                packet_features['src_port'] = packet[TCP].sport
                packet_features['dst_port'] = packet[TCP].dport

                # Check if the packet contains payload (Raw layer)
                if Raw in packet:
                    raw_payload = packet[Raw].load
                    packet_features['payload_size'] = len(raw_payload)

                    # Clean the payload contents: replace non-printable characters with a placeholder or remove them
                    printable = set(string.printable)
                    cleaned_payload = ''.join(filter(lambda x: x in printable, raw_payload.decode('utf-8', errors='ignore')))
                    packet_features['payload_contents'] = cleaned_payload

        # Append the dictionary of features to the list
        payload_characteristics_features.append(packet_features)


    return pd.DataFrame(payload_characteristics_features)


**Download the datasets if not present locally:**

In [ ]:
def download_dataset(filename, url) :  

    #if the file is not present locally, let download it!
    if os.path.isfile(filename) == False:
        print("Downloading "+filename+" at the url "+url+"/n")
        wget.download(url, filename)
    else:
        print(filename+" already exists. We use the local copy\n")

    return(filename)


**Main Execution and Saving DataFrames:**

In [ ]:
if __name__ == "__main__":
    pcap_file_path = download_dataset('datasets/output_part_1.pcap', 'https://fabrice.theoleyre.cnrs.fr/tmp/dataset/output_part_1.pcap') 
    packets = load_pcap_data(pcap_file_path)

    network_traffic_df = extract_network_traffic_features(packets)
    network_traffic_df.to_csv('/content/drive/MyDrive/network_traffic_features.csv', index=False)

    session_behavior_df = extract_session_behavior_features(packets)
    session_behavior_df.to_csv('/content/drive/MyDrive/session_behaviour_features.csv', index=False)

    network_flow_df = extract_network_flow_features(packets)
    network_flow_df.to_csv('/content/drive/MyDrive/network_flow_features.csv', index=False)

    protocol_specific_df = extract_protocol_specific_features(packets)
    protocol_specific_df.to_csv('/content/drive/MyDrive/protocol_specific_features.csv', index=False)

    payload_characteristics_df = extract_payload_characteristics_features(packets)
    payload_characteristics_df.to_csv('/content/drive/MyDrive/payload_characteristics_features.csv', index=False)
